In [1]:
cd C:\Users\LIMDAESUN\Desktop\카이스트관련\투자론

C:\Users\LIMDAESUN\Desktop\카이스트관련\투자론


In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import statsmodels.api as sm

In [3]:
rf = (pd.read_excel('rf.xlsx', index_col=0)/100).resample('D').last().interpolate(method = 'cubic')
price = pd.read_excel('코스피개별가격데이터.xlsx', index_col=0)
KOSPI = pd.read_excel('코스피지수.xlsx', index_col = 0)
ret = np.log(price).diff().iloc[1:]
my_index = sorted(list(set(ret.index)&set(rf.index)))
KOSPI_ret = np.log(KOSPI).diff().iloc[1:].rename(columns = {'코스피지수' : '코스피수익률'}).loc[my_index]
rf = rf.loc[my_index]
ret = ret.loc[my_index]

In [4]:
my_port = ['A005930','A005380','A000660','A068270','A005490','A002420']

 ## Optimized Weight by std minimizer

In [5]:
def w_by_std_min(my_port_list, ret, a = 0.05) :
    my_port = my_port_list
    my_fun_str = ''
    for i in range(len(my_port)) :
        my_fun_str += '-x[' + str(i) + ']'
    cons = ({'type':'eq', 'fun':lambda x : eval(my_fun_str) +1})
    if len(my_port) >= 4 :
        bnds = tuple([(0,0.3)] * len(my_port))
    else :
        bnds = tuple([(0,1)] * len(my_port))
    def objfun(x, ret, alpha) :
        cov = ret[my_port].ewm(alpha = a).cov().loc[ret.index[-1]]
        return np.sqrt(x.reshape(1,-1).dot(cov).dot(x.reshape(-1,1)))
    x0 = tuple([1/len(my_port)]*len(my_port))
    res = minimize(objfun, x0, method = 'SLSQP', bounds = bnds, constraints = cons, args= (ret, a))
    return pd.DataFrame(res.x, index = my_port, columns = ['Wegiht']).T

In [6]:
w_by_std_min(my_port, ret)

,A005930,A005380,A000660,A068270,A005490,A002420
Wegiht,0.218264,0.243214,0.120564,0.088876,0.3,0.029083


 ## Optimized Weight by Maximum Sharp

In [7]:
def w_by_sharp_max(my_port_list, ret, KOSPI_ret, a = 0.05) :
    my_port = my_port_list
    beta_lst = []
    for i in my_port :
        y = ret[my_port][i]
        x1 = KOSPI_ret
        x = sm.add_constant(x1)
        result = sm.OLS(y,x).fit()
        beta = result.params[1]
        beta_lst.append(beta)
    rm = (1 + KOSPI_ret[KOSPI_ret.columns[0]].resample('M').last().mean())**12-1
    risk_free = rf.RF.mean()        
    Required_Return = risk_free + np.array(beta_lst) * (rm - risk_free)
    
    my_fun_str = ''
    for i in range(len(my_port)) :
        my_fun_str += '-x[' + str(i) + ']'
    cons = ({'type':'eq', 'fun':lambda x : eval(my_fun_str) +1})
    if len(my_port) >= 4 :
        bnds = tuple([(0,0.3)] * len(my_port))
    else :
        bnds = tuple([(0,1)] * len(my_port))
        
    def objfun(x, ret, alpha) :
        cov = ret[my_port].ewm(alpha = a).cov().loc[ret.index[-1]]
        PF_std =  np.sqrt(x.reshape(1,-1).dot(cov).dot(x.reshape(-1,1)))
        PF_ret = (x * Required_Return).sum()
        sharp = PF_ret/PF_std
        return -sharp
    x0 = tuple([1/len(my_port)]*len(my_port))
    res = minimize(objfun, x0, method = 'SLSQP', bounds = bnds, constraints = cons, args= (ret, a))
    return pd.DataFrame(res.x, index = my_port, columns = ['Wegiht']).T

In [8]:
w_by_sharp_max(my_port, ret, KOSPI_ret, a = 0.05)

C:\Users\LIMDAESUN\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,A005930,A005380,A000660,A068270,A005490,A002420
Wegiht,0.3,0.23716,0.036341,0.08417,0.290636,0.051692
